In [1]:
#カーネル2.7で動かす必要あり
!python -V


Python 3.5.2 :: Anaconda 4.2.0 (x86_64)


In [1]:
# -*- coding: shift_jis -*-
# capacity : 各窓口の配車台数
# rate     : 物件×窓口の成約率マトリクス
# x        : 物件×窓口の配車リスト(変数)
# sum_{i,j} rate_{i,j} * x_{i,j} : 予測成約台数(最適化目的関数)
#使い方：引数に〜を入れる

from ortools.linear_solver import pywraplp
import csv,re,sys

In [16]:
k = 0
date = ""
# ・yyyymmdd_list.csv #前工程で出した、今回仕入れた車のrateを食わせる
# ・yyyymmdd_settei.txt #
# ・amount.csv #
ratefile = "/Users/01017387/Desktop/しごと/コード/03_配車最適化/03_analysis/03_最適化/03_最適化/20161101_list.csv"
date= "20161121_test"
capacityfile = "/Users/01017387/Desktop/しごと/コード/03_配車最適化/03_analysis/03_最適化/03_最適化/amount.csv"
setfile = "/Users/01017387/Desktop/しごと/コード/03_配車最適化/03_analysis/03_最適化/03_最適化/20161101_settei.txt"
optMode = "1"
# 1 : 成約台数最大化
# 2 : 最小成約達成の最公平
# 3 : 公平性をペナルティとした成約台数最大化

# set limit time (ms)
timeLimit = 300000

solverType = {
    'IP':pywraplp.Solver.BOP_INTEGER_PROGRAMMING,
    'MIP':pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING,
    'LP':pywraplp.Solver.GLOP_LINEAR_PROGRAMMING
    }

# 1.define parameter

def SetupFile(ratefile, capacityfile, setfile):

    f = open(setfile)
    data = f.read()
    f.close()
    setlist = data.split()

    s = ''
    k = 0

    for item in setlist:
        if item[len(item)-1].isdigit():
            s = s + item
            if '=' not in s:
                numPlace = min([i for i in range(len(s)) if s[i].isdigit()])
                s = ''.join(s[j] for j in range(numPlace)) + '=' + ''.join(s[m] for m in range(len(s)) if m >= numPlace)
            exec(s) in locals()
            s = ''
            k = k + 1
        else:
            s = s + item
    assert k == 3, "Three parameter (num_car, num_mado, pred) should be set"

    rate = [[ 0.0 for j in range(num_mado)] for i in range(num_car)]
    capacity = [0]*num_mado

    csvobj = csv.reader( open(ratefile,"rb"), delimiter=',' )
    y = [ v for v in csvobj]
    #print(len(rate))
    #print(len(y))
    #print(len(y[0]))
    #print(len(rate[0]))
    #print(num_car)
    #print(num_mado)
    for i in range(num_car):
        for j in range(num_mado):
            rate[i][j] = float(y[i][j])
    csvobj = csv.reader(open(capacityfile,"rb"), delimiter=',' )
    y = [v for v in csvobj]
    for i in range(num_mado):
        capacity[i] = int(y[i][0])

    return {'num_car':num_car,'num_mado':num_mado,'pred':pred,'rate':rate,'capacity':capacity}


setRes = SetupFile(ratefile, capacityfile, setfile)

num_car = setRes['num_car']
num_mado = setRes['num_mado']
seiyaku_least = setRes['pred']
rate = setRes['rate']
capacity = setRes['capacity']

res  = [[ 0.0 for j in range(num_mado)] for i in range(num_car)]

# alpha：各店舗の理想成約率
irate = float( 1.0 / num_mado )
alpha = [irate] * num_mado

# 2.define solver（問題の選択）
#(i).整数
#solver = pywraplp.Solver('Transfer',pywraplp.Solver.BOP_INTEGER_PROGRAMMING)
#(ii).整数実数混合
#solver = pywraplp.Solver('Transfer',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#(iii).実数
#solver = pywraplp.Solver('Transfer',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
solver = pywraplp.Solver('Transfer',solverType['MIP'])

# 4.define variables
x = [[0 for mado in range(num_mado)] for car in range(num_car)]
for i in range(num_car):
    for j in range(num_mado):
        x[i][j] = solver.IntVar(0,1, name="x(%s,%s)"%(i,j))

# 成約率の公平さ
t = [0 for mado in range(num_mado)]
for i in range(num_mado):
    t[i] = solver.NumVar(0,solver.infinity(), name="t(%s)"%i )

# 最大成約率と最小成約率
u = solver.NumVar(0, solver.infinity(), name="upper")
l = solver.NumVar(0, solver.infinity(), name="lower")
solver.Add(l <= u)

# 5.define constraints(制約条件の設定)

# 物件の数の固定（同じ物件の配車は一回だけ）
for i in range(num_car):
    #solver.Add(solver.Sum(x[i][j] for j in range(num_mado)) == 1 ) #demand[i])#仕入れ車が制約台数より同じか少ない場合
    solver.Add(solver.Sum(x[i][j] for j in range(num_mado)) <= 1 ) #demand[i])#仕入れ車が、制約台数より多い場合

# 各窓口の最大配車台数の制限を追加
for j in range(num_mado):
    #solver.Add(solver.Sum(x[i][j] for i in range(num_car)) == capacity[j] )
    solver.Add(solver.Sum(x[i][j] for i in range(num_car)) <= capacity[j] )

mado_available = 0
for j in range(num_mado):
    if capacity[j] > 0:
        solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car)) <= u * capacity[j])
        solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car)) >= l * capacity[j])
        mado_available = mado_available + 1

# set limit time (ms)
solver.SetTimeLimit(timeLimit)
# solver.SetTimeLimit(30)

# 6.予測成約台数の最大化
def MaxSeiyaku():
    solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)))

# 7. 最公平
def Saikouhei():
    #print num_mado
    print "Checking feasibility of the least seiyaku..."
    solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)))
    solver.Solve()
    assert sum(rate[i][j]*x[i][j].solution_value() for i in range(num_car) for j in range(num_mado)) >= seiyaku_least, "Max value is small than seiyaku_least!!!"
    print "Least seiyaku is checked"
    print "Adding additional Constraints..."
    for k in range(num_mado):
        if capacity[k] > 0:
        #solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car))-alpha[j]*solver.Sum(rate[i][j] * x[i][j] for i in range(num_car) for j in range(num_mado)) >= -1.0 * t[j] )
        #solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car))-alpha[j]*solver.Sum(rate[i][j] * x[i][j] for i in range(num_car) for j in range(num_mado)) <=  1.0 * t[j] )
            solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car))-alpha[k]*solver.Sum(rate[i][j] * x[i][j] for i in range(num_car) for j in range(num_mado)) >= -1.0 * t[k] * capacity[k] )
            solver.Add(solver.Sum(rate[i][j] * x[i][j] for i in range(num_car))-alpha[k]*solver.Sum(rate[i][j] * x[i][j] for i in range(num_car) for j in range(num_mado)) <=  1.0 * t[k] * capacity[k] )
    solver.Add(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)) >= seiyaku_least)
    solver.Minimize(solver.Sum(t[i] for i in range(num_mado)))

# 8. Multiple Objective
def Multiobj():
    solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)) - (u - l) * mado_available / 2)
    #solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)) - (u - l) * mado_available)
    #solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)) - (u - l) * num_car)
    #solver.Maximize(solver.Sum(rate[i][j]*x[i][j] for i in range(num_car) for j in range(num_mado)) - (u - l))

objSet = {'1':MaxSeiyaku,'2':Saikouhei,'3':Multiobj}

print("Setting Objective")

objSet[optMode]()

print "Ready...GO!"
print "time limit: " + str(timeLimit/1000) + "s"

result = solver.Solve()

def PrintOutput():
    print 'total car', num_car
    print 'mado available', mado_available
    print 'mado upper rate', u.solution_value()
    print 'mado lower rate', l.solution_value()
    print 'least seiyaku', seiyaku_least
    print 'total seiyaku', sum(rate[i][j]*x[i][j].solution_value() for i in range(num_car) for j in range(num_mado))
    print 'Iterations: ', solver.Iterations()
    print 'Time: ', solver.WallTime(),"ms"

def ExportResult():
    for i in range(num_car):
        for j in range(num_mado):
            res[i][j] = x[i][j].solution_value()

    filename = "/Users/01017387/Desktop/jupyter/" + date + "_result.csv"
    print(filename)
    f = open(filename,"wb")
    dataWriter = csv.writer(f)
    dataWriter.writerows(res)
    f.close()

Setting Objective
Ready...GO!
time limit: 300s


In [17]:
if result == solver.OPTIMAL:
    print "done"
    PrintOutput()
    ExportResult()

else:
    if result == solver.FEASIBLE:
        print 'A potentially suboptimal solution was found.'
        PrintOutput()
        ExportResult()
    else:
        print 'The solver could not solve the problem.'
        print 'Time: ', solver.WallTime(),"ms"


done
total car 18
mado available 4
mado upper rate 0.515346661012
mado lower rate 0.0
least seiyaku 3
total seiyaku 7.22412106452
Iterations:  0
Time:  55 ms
/Users/01017387/Desktop/jupyter/20161121_test_result.csv


In [16]:
#18台のうち、7.2台が１ヶ月以内に成約すると見込まれる
# !cat 20161121_test_result.csv
import pandas as pd
da = pd.read_csv("20161121_test_result.csv",names=["aa","bb","cc","dd"])
db = pd.read_csv("/Users/01017387/Desktop/しごと/コード/03_配車最適化/03_analysis/03_最適化/03_最適化/20161101_list.csv",names=["a","b","c","d"])
dc = pd.concat([da,db],axis=1)
dc
#配車台数の成約付き問題を解いている
# 合計の確率が最も高くなるように解を求める

,aa,bb,cc,dd,a,b,c,d
0,0.0,0.0,0.0,1.0,0.352178,0.274995,0.174699,0.259743
1,0.0,1.0,0.0,0.0,0.428311,0.343284,0.225840,0.325947
2,0.0,0.0,0.0,1.0,0.332814,0.258183,0.162643,0.243550
3,0.0,1.0,0.0,0.0,0.398661,0.316264,0.205176,0.299669
4,1.0,0.0,0.0,0.0,0.605703,0.517786,0.375955,0.499955
5,0.0,1.0,0.0,0.0,0.405262,0.322232,0.209691,0.305464
6,1.0,0.0,0.0,0.0,0.517993,0.428956,0.296496,0.411573
7,0.0,1.0,0.0,0.0,0.437315,0.351601,0.232318,0.334058
8,0.0,1.0,0.0,0.0,0.393962,0.312033,0.201992,0.295564
9,1.0,0.0,0.0,0.0,0.447583,0.361571,0.241133,0.345267
